In [1]:
# ruff: noqa
import os

os.sys.path.append("..")
from feedback_grape.fgrape import optimize_pulse_with_feedback
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
    cosm,
    sinm,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

## defining parameterized operations that are repeated num_time_steps times

In [2]:
N_cav = 5

In [3]:
def qubit_unitary(alpha):
    """
    TODO: see if alpha, can be sth elser other than scalar, and if the algo understands this
    see if there can be multiple params like alpha and beta input
    """
    return expm(
        -1j
        * (
            alpha * tensor(identity(N_cav), sigmap())
            + alpha.conjugate() * tensor(identity(N_cav), sigmam())
        )
        / 2
    )

In [4]:
def qubit_cavity_unitary(beta):
    return expm(
        -1j
        * (
            beta
            * (
                tensor(destroy(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmap())
            )
            + beta.conjugate()
            * (
                tensor(create(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmam())
            )
        )
        / 2
    )

In [5]:
alpha = 0.1 + 0.1j
beta = 0.1 + 0.1j
Uq = qubit_unitary(alpha)
Uqc = qubit_cavity_unitary(beta)
print(
    "Uq unitary check:",
    jnp.allclose(Uq.conj().T @ Uq, jnp.eye(Uq.shape[0]), atol=1e-7),
)
print(
    "Uqc unitary check:",
    jnp.allclose(Uqc.conj().T @ Uqc, jnp.eye(Uqc.shape[0]), atol=1e-7),
)

Uq unitary check: True
Uqc unitary check: True


In [6]:
qubit_unitary(0.1)

Array([[0.99875027+0.j        , 0.        -0.04997917j,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        -0.04997917j, 0.99875027+0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.99875027+0.j        , 0.        -0.04997917j,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        -0.04997917j, 0.99875027+0.j        ,
        0.        +0.j        , 0.        +0.

In [7]:
from feedback_grape.utils.operators import create, destroy


def povm_measure_operator(measurement_outcome, gamma, delta):
    """
    POVM for the measurement of the cavity state.
    returns Mm ( NOT the POVM element Em = Mm_dag @ Mm ), given measurement_outcome m, gamma and delta
    """
    number_operator = tensor(create(N_cav) @ destroy(N_cav), identity(2))
    angle = (gamma * number_operator) + delta / 2
    meas_op = jnp.where(
        measurement_outcome == 1,
        cosm(angle),
        sinm(angle),
    )
    return meas_op

### defining initial (thermal) state

In [8]:
# initial state is a thermal state coupled to a qubit in the ground state?
n_average = 1
# natural logarithm
beta = jnp.log((1 / n_average) + 1)
diags = jnp.exp(-beta * jnp.arange(N_cav))
normalized_diags = diags / jnp.sum(diags, axis=0)
rho_cav = jnp.diag(normalized_diags)

In [9]:
rho_cav.shape

(5, 5)

In [10]:
rho0 = tensor(rho_cav, basis(2, 0) @ basis(2, 0).conj().T)

In [11]:
from feedback_grape.fgrape import _probability_of_a_measurement_outcome_given_a_certain_state
_probability_of_a_measurement_outcome_given_a_certain_state(rho0, 1, povm_measure_operator, [0.1, -3*jnp.pi / 2])

Array(0.88501454, dtype=float64)

### defining target state

In [12]:
psi_target = tensor((fock(N_cav, 1) + fock(N_cav, 2) + fock(N_cav, 3)) / jnp.sqrt(3), basis(2))

rho_target = psi_target @ psi_target.conj().T
rho_target.shape

(10, 10)

In [13]:
from feedback_grape.utils.fidelity import fidelity
print(fidelity(U_final=rho0, C_target=rho_target, type="density"))

0.3880207144091314


### initialize random params

In [14]:
num_time_steps = 5
num_of_iterations = 1000
learning_rate = 0.05
# avg_photon_numer = 2 When testing kitten state


initial_params = {
    "POVM": {
        "gamma": jnp.pi / 3,
        "delta": jnp.pi / 3,
    },
    "U_q": {
        "alpha": jnp.pi / 3,
    },
    "U_qc": {
        "beta": jnp.pi / 3,
    }
}


result = optimize_pulse_with_feedback(
    U_0=rho0,
    C_target=rho_target,
    parameterized_gates=[
        povm_measure_operator,
        qubit_unitary,
        qubit_cavity_unitary,
    ],
    measurement_indices=[0],
    initial_params=initial_params,
    num_time_steps=num_time_steps,
    mode="nn",
    goal="fidelity",
    optimizer="adam",
    max_iter=num_of_iterations,
    convergence_threshold=1e-20,
    learning_rate=learning_rate,
    type="density",
)

In [15]:
result

FgResult(optimized_rnn_parameters={'params': {'Dense_0': {'bias': Array([-0.04012918,  0.21233846,  0.41916883,  0.7741086 ], dtype=float32), 'kernel': Array([[-6.48014247e-02,  1.13182589e-01, -1.27241939e-01,
        -2.24849656e-01],
       [ 9.29016806e-03,  2.46264398e-01,  7.05307782e-01,
         6.12710595e-01],
       [ 3.46890017e-02, -2.14393660e-01,  1.77045369e+00,
         1.26670516e+00],
       [-5.85440286e-02,  1.07428104e-01,  4.49466646e-01,
        -2.35529631e-01],
       [-3.43436860e-02, -4.05002803e-01, -6.53316557e-01,
        -6.03038557e-02],
       [-5.50922193e-02, -5.60203455e-02, -2.00591654e-01,
         4.02358145e-01],
       [ 4.32903394e-02, -2.11749986e-01,  1.33660346e-01,
        -1.66298211e+00],
       [ 2.07558833e-02,  4.58290242e-02,  5.91896117e-01,
        -1.82525799e-01],
       [-4.83784679e-04,  1.30473018e-01,  6.82662368e-01,
        -7.71707356e-01],
       [-6.53931499e-02,  1.99147686e-02, -2.04631060e-01,
         6.09878898e-02]

In [16]:
print(result.final_purity)

None


In [17]:
print(result.final_fidelity)

0.8133550660337464


In [18]:
print(result.iterations)

1000


In [19]:
print(result.arr_of_povm_params)

[[[1.0471975511965976, 1.0471975511965976], [1.0471975511965976], [1.0471975511965976]], [[-0.0016242020812901811, -0.0032900861287659955], [1.4429755442483019], [3.3352049908164054]], [[-0.0005666572743259005, 0.6210883066382602], [3.6493352208265772], [2.570416137388485]], [[0.00037475809769982327, 0.629152136784469], [-1.7720907365528995], [-3.675815234775065]], [[-0.003429879232455718, 0.626701223807564], [-0.5435530235711684], [4.5669562827207475]]]


In [20]:
print(result.arr_of_povm_params[1])

[[-0.0016242020812901811, -0.0032900861287659955], [1.4429755442483019], [3.3352049908164054]]
